In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='default', category=FutureWarning)

import glob
import numpy as np
import os
from pathlib import Path


import cv2 as cv
import cv2

import blend_modes

from IPython.display import display

import ipyplot

from PIL import Image as Img
from PIL import ImageChops, ImageOps,  ImageEnhance

from colour import Color
import webcolors

import block_meta as blmt

from ruined import ruin_blocks, reg_rubble, reg_variant
from ruined import concretes_names, concretes, stones_names, stones, etces_names, etces, concrete_colors

from color_utils import normalize_lum, get_color_ramp, colour_from_rgb_tuple, colorize_overlay

from texture_generation import get_rgba, get_mask, img_blend, composite
from texture_generation import big_brick, big_brick_grout, big_brick_3d, big_brick_grout_lighter
from texture_generation import small_brick, sm_brick_grout, sm_brick_3d
from texture_generation import specks_1, specks_2, mellow_noise, contrasty_noise
from texture_generation import moss_trans1, moss_trans2, moss_a, moss_b, moss_c

mod_name = "lostruins"
block_textures_dir = Path(f"assets_out/{mod_name}/textures/block/")

con_bricks = glob.glob(os.path.join(f"assets_out/{mod_name}/textures/block/concretes","*_big_bricks.png"))
bbricks = [Img.open(path) for path in con_bricks]
bbricks_names = [texture.split('/')[-1].split('.')[0] for texture in con_bricks]



for dir_name in ["concretes", 'etc', 'roads','stones']:
    Path(block_textures_dir / dir_name).mkdir(parents=True, exist_ok=True)

In [10]:
# moved to colorutil.py

# def rgb_to_hex(rgb, with_hash=None):
    
#     der_hex = '%02x%02x%02x' % rgb
#     if with_hash:
#         der_hex =  f"#{der_hex}"
#     return der_hex

# def normalize_lum(low, mid, high):
#     #print("IN =>\t", low, "\t", mid, "\t", high)

#     if mid > high:
#         high = .9
#     if mid < low: 
#         low = .05
    
#     r_high = mid + ((high - mid) / 1.2)
#     r_mid = mid
#     r_low = (mid - low) / 2 
    
#     #print("out=>\t",r_low, "\t", r_mid, "\t", r_high, "\n")
#     #print(f"texture luminance = {mid}")
#     return ( r_low, r_mid, r_high)

# def get_color_ramp(steps:int, 
#                    base_hex:str=None, 
#                    base_rgb:tuple=None, 
#                    light_lum=0.95, 
#                    dark_lum= 0.15):
    
#     if base_rgb:
#         r,g,b = base_rgb
#         c = Color(rgb=(r/255, g/255, b/255))
#     if base_hex and not base_rgb:
#         c = Color(base_hex)
#     if not base_hex and not base_rgb:
#         return False
    
#     low, mid, high = normalize_lum(dark_lum, c.get_luminance(), light_lum)
#     # print(low, mid, high )
#     light_tone = Color(hue=c.get_hue(), saturation=c.get_saturation(), luminance=high)
#     dark_tone = Color(hue=c.get_hue(), saturation=c.get_saturation(), luminance=low)

#     if base_rgb:
#         ramp_list = [tone.get_rgb() for tone in dark_tone.range_to(light_tone, steps)]
#         # print("_b, _w",_b, _w)
#         _ramp_list = [(int(_r * 255), int(_g * 255), int(_b * 255)) for (_r,_g,_b) in ramp_list]
#         # print(cd.get_luminance(), "\t", c.get_luminance(), "\t",cl.get_luminance())
        
#         return tuple(_ramp_list)
#     else:
#         return tuple([color.get_web() for color in cd.range_to(cl, 2)])


# def colour_from_rgb_tuple(rgb_tuple:tuple):
#     r,g,b = rgb_tuple
#     return Color(rgb=(r/255, g/255, b/255))


# def colorize_overlay(pil_img, dark_rgb, light_rgb, mid_rgb=None):
#     # we need to put the alpha channel aside
#         _r, _g_, _b, im_alpha = pil_img.split()
#         if mid_rgb:
#             # image needs to be converted to grayscale for color ramp
#             im_in_color = ImageOps.colorize(pil_img.convert('L'), 
#                                             dark_rgb, light_rgb, mid=mid_rgb)
#         else:
#             im_in_color = ImageOps.colorize(im_alpha.convert('L'), dark_rgb, light_rgb)
#             # add alpha channel back to colorized image
        
#         im_in_color.putalpha(im_alpha)
#         return im_in_color
    
        
            
            

    

In [7]:
file_ends = [numb if numb > 0 else "" for numb in range(6)]

cracks = []
darker_cracks = []
full_block_cracks = []
big_brick_cracks = []

for numb in range(6):
    file_end = file_ends[numb]
    cracks.append(Img.open(f'res16/cracks{file_end}.png'))
    darker_cracks.append(Img.open(f'res16/cracks_darker/cracks{file_end}.png'))
    full_block_cracks.append(Img.open(f'res16/cracks_full_block/cracks{file_end}.png'))
    big_brick_cracks.append(Img.open(f'res16/cracks_big_brick/cracks{file_end}.png'))
cracker = cracks[5]

bn_out, bi_out = ([],[])
cracknames_out, cracks_out = ([],[])
chipn_out, chips_out = ([],[])

def add_it(pil_img, name, img_out_list=bi_out, name_out_list=bn_out):
    img_out_list.append(pil_img)
    name_out_list.append(name)
    
#for name, brick in dict(zip(bbricks_names, bbricks)).items():
for name, brick in dict(zip(concretes_names, concretes)).items():
    color = name.removesuffix('_concrete')
    if len(name.split('_concrete')[1]):
        #color = name.removesuffix('_concrete_big_bricks')
        basename = f"{color}_concrete"
    else:
        
        basename = name
    #print(name)
    rgb = blmt.BLOCK_META[basename]['rgb']
    b_colour = colour_from_rgb_tuple(rgb)
    b, w = get_color_ramp(base_rgb=rgb, steps=2, light_lum=.99, dark_lum=0.15)
    b_lum = b_colour.get_luminance()
    
    
    add_it(b, f'{name} dark', chips_out, chipn_out)
    add_it(w, f'{name} light', chips_out, chipn_out)
    
    use_darker_cracks = ['black']
    add_it(brick, f"{color} {rgb}")
    
    for numb in range(6):
        file_end = file_ends[numb]

#         cracked = img_blend(brick, cracks[numb], blend_modes.grain_merge, .75)
#         cracked2 = img_blend(brick, cracks[numb], blend_modes.addition, .1)
        colored_original_crack = cracks[numb]
        colored_darker_crack =  darker_cracks[numb]
        colored_full_block_crack = full_block_cracks[numb]
        colored_big_brick_crack = big_brick_cracks[numb]
        
        plain_full_block_crack = full_block_cracks[numb]
        plain_big_brick_crack = colored_big_brick_crack
        r3, g3, b3, fbc_alpha = plain_full_block_crack.split()
        r3, g3, b3, bbc_alpha = plain_full_block_crack.split()
        
        colored_original_crack = colorize_overlay(colored_original_crack,  b,  w, rgb)
        colored_darker_crack =  colorize_overlay(colored_darker_crack,  b,  w, mid_rgb=rgb)
        colored_full_block_crack = colorize_overlay(colored_full_block_crack,  b,  w, mid_rgb=rgb)
        colored_big_brick_crack = colorize_overlay(colored_big_brick_crack,  b,  w, mid_rgb=rgb)
        
        colored_original_crack.save(f'res16/cracks_colors/original/{name}_cracks{file_end}.png')
        colored_darker_crack.save(f'res16/cracks_colors/darker/{name}_cracks{file_end}.png')
        colored_full_block_crack.save(f'res16/cracks_colors/full_block/{name}_cracks{file_end}.png')
        colored_big_brick_crack.save(f'res16/cracks_colors/big_brick/{name}_cracks{file_end}.png')
        
        add_it(colored_big_brick_crack, f'{name}_cracks_big_brick{file_end}',
              cracks_out, cracknames_out)
        
        
        
        cracked_brick = brick.copy()
        #cracked_brick = img_blend(cracked_brick, cracked_brick, blend_modes.multiply, .9)
        cracked_brick.paste(colored_full_block_crack, (0,0), colored_full_block_crack)
        cracked_brick2 = cracked_brick.copy()
        cracked_brick2.paste(plain_full_block_crack, (0,0), plain_full_block_crack)
        # just the cracks, mamm
        
        if color in ['black', 'blue', 'brown', 'purple', 'red', 'gray']:
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .25)
        
        elif color in ['green', 'gray']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .30)
        
        elif color in ['cyan', 'green','light_blue', 'magenta', 'orange', 'pink', 'lime']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .45)
        
        elif color in ['pink', 'yellow', 'light_gray']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .95)
        else:
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.darken_only, .35)
            #cracked_brick = cracked_brick2
        
        add_it(cracked_brick, f"{name}_cracked{file_end}")
        
        cracked_brick.save(f"assets_out/{mod_name}/textures/block/concretes/{name}_cracked{file_end}.png")
        
        #add_it(cracked_brick2, f"2{name}_cracked{file_end}")
        #add_it(big_brick(cracked_brick, big_brick_grout), f"{name}_cracked_big_bricks{file_end}")
    
ipyplot.plot_class_representations(bi_out, bn_out)

In [8]:
bn_out, bi_out = ([],[])
cracknames_out, cracks_out = ([],[])
chipn_out, chips_out = ([],[])

def add_it(pil_img, name, img_out_list=bi_out, name_out_list=bn_out):
    img_out_list.append(pil_img)
    name_out_list.append(name)

#for name, brick in dict(zip(bbricks_names, bbricks)).items():
for name, brick in dict(zip(stones_names, stones)).items():
    color = name
    basename = name
    tex_dir = stones
    #print(name)
    rgb = blmt.BLOCK_META[basename]['rgb']
    b_colour = colour_from_rgb_tuple(rgb)
    b, w = get_color_ramp(base_rgb=rgb, steps=2, light_lum=.99, dark_lum=0.15)
    b_lum = b_colour.get_luminance()
    
    
    add_it(b, f'{name} dark', chips_out, chipn_out)
    add_it(w, f'{name} light', chips_out, chipn_out)
    
    use_darker_cracks = ['black']
    add_it(brick, f"{color} {rgb}")
    
    for numb in range(6):
        file_end = file_ends[numb]

#         cracked = img_blend(brick, cracks[numb], blend_modes.grain_merge, .75)
#         cracked2 = img_blend(brick, cracks[numb], blend_modes.addition, .1)
        colored_original_crack = cracks[numb]
        colored_darker_crack =  darker_cracks[numb]
        colored_full_block_crack = full_block_cracks[numb]
        colored_big_brick_crack = big_brick_cracks[numb]
        
        plain_full_block_crack = full_block_cracks[numb]
        plain_big_brick_crack = colored_big_brick_crack
        r3, g3, b3, fbc_alpha = plain_full_block_crack.split()
        r3, g3, b3, bbc_alpha = plain_full_block_crack.split()
        
        colored_original_crack = colorize_overlay(colored_original_crack,  b,  w, rgb)
        colored_darker_crack =  colorize_overlay(colored_darker_crack,  b,  w, mid_rgb=rgb)
        colored_full_block_crack = colorize_overlay(colored_full_block_crack,  b,  w, mid_rgb=rgb)
        colored_big_brick_crack = colorize_overlay(colored_big_brick_crack,  b,  w, mid_rgb=rgb)
        
        colored_original_crack.save(f'res16/cracks_colors/original/{name}_cracks{file_end}.png')
        colored_darker_crack.save(f'res16/cracks_colors/darker/{name}_cracks{file_end}.png')
        colored_full_block_crack.save(f'res16/cracks_colors/full_block/{name}_cracks{file_end}.png')
        colored_big_brick_crack.save(f'res16/cracks_colors/big_brick/{name}_cracks{file_end}.png')
        
        add_it(colored_big_brick_crack, f'{name}_cracks_big_brick{file_end}',
              cracks_out, cracknames_out)
        
        
        
        cracked_brick = brick.copy()
        #cracked_brick = img_blend(cracked_brick, cracked_brick, blend_modes.multiply, .9)
        cracked_brick.paste(colored_full_block_crack, (0,0), colored_full_block_crack)
        cracked_brick2 = cracked_brick.copy()
        cracked_brick2.paste(plain_full_block_crack, (0,0), plain_full_block_crack)
        # just the cracks, mamm
        
        if color in ['black', 'blue', 'brown', 'purple', 'red', 'gray']:
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .25)
        
        elif color in ['green', 'gray']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .30)
        
        elif color in ['cyan', 'green','light_blue', 'magenta', 'orange', 'pink', 'lime']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .45)
        
        elif color in ['pink', 'yellow', 'light_gray']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .95)
        else:
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.darken_only, .35)
            #cracked_brick = cracked_brick2
        
        add_it(cracked_brick, f"{name}_cracked{file_end}")
        
        cracked_brick.save(f"assets_out/{mod_name}/textures/block/stones/{name}_cracked{file_end}.png")
        
        #add_it(cracked_brick2, f"2{name}_cracked{file_end}")
        #add_it(big_brick(cracked_brick, big_brick_grout), f"{name}_cracked_big_bricks{file_end}")
    
ipyplot.plot_class_representations(bi_out, bn_out)

In [9]:
bn_out, bi_out = ([],[])
cracknames_out, cracks_out = ([],[])
chipn_out, chips_out = ([],[])

def add_it(pil_img, name, img_out_list=bi_out, name_out_list=bn_out):
    img_out_list.append(pil_img)
    name_out_list.append(name)

#for name, brick in dict(zip(bbricks_names, bbricks)).items():
for name, brick in dict(zip(etces_names, etces)).items():
    if name in ['gray_wood_boards', 'gray_wood_planks_nailed', 'gray_wood_planks']:
        continue
    color = name
    basename = name
    #print(name)
    rgb = blmt.BLOCK_META[basename]['rgb']
    b_colour = colour_from_rgb_tuple(rgb)
    b, w = get_color_ramp(base_rgb=rgb, steps=2, light_lum=.99, dark_lum=0.15)
    b_lum = b_colour.get_luminance()

    
    add_it(b, f'{name} dark', chips_out, chipn_out)
    add_it(w, f'{name} light', chips_out, chipn_out)
    
    use_darker_cracks = ['black']
    add_it(brick, f"{color} {rgb}")
    
    for numb in range(6):
        file_end = file_ends[numb]
        
#         cracked = img_blend(brick, cracks[numb], blend_modes.grain_merge, .75)
#         cracked2 = img_blend(brick, cracks[numb], blend_modes.addition, .1)
        colored_original_crack = cracks[numb]
        colored_darker_crack =  darker_cracks[numb]
        colored_full_block_crack = full_block_cracks[numb]
        colored_big_brick_crack = big_brick_cracks[numb]
        
        plain_full_block_crack = full_block_cracks[numb]
        plain_big_brick_crack = colored_big_brick_crack
        r3, g3, b3, fbc_alpha = plain_full_block_crack.split()
        r3, g3, b3, bbc_alpha = plain_full_block_crack.split()
        
        colored_original_crack = colorize_overlay(colored_original_crack,  b,  w, rgb)
        colored_darker_crack =  colorize_overlay(colored_darker_crack,  b,  w, mid_rgb=rgb)
        colored_full_block_crack = colorize_overlay(colored_full_block_crack,  b,  w, mid_rgb=rgb)
        colored_big_brick_crack = colorize_overlay(colored_big_brick_crack,  b,  w, mid_rgb=rgb)
        
        colored_original_crack.save(f'res16/cracks_colors/original/{name}_cracks{file_end}.png')
        colored_darker_crack.save(f'res16/cracks_colors/darker/{name}_cracks{file_end}.png')
        colored_full_block_crack.save(f'res16/cracks_colors/full_block/{name}_cracks{file_end}.png')
        colored_big_brick_crack.save(f'res16/cracks_colors/big_brick/{name}_cracks{file_end}.png')
        
        add_it(colored_big_brick_crack, f'{name}_cracks_big_brick{file_end}',
              cracks_out, cracknames_out)
        
        
        
        cracked_brick = brick.copy()
        #cracked_brick = img_blend(cracked_brick, cracked_brick, blend_modes.multiply, .9)
        cracked_brick.paste(colored_full_block_crack, (0,0), colored_full_block_crack)
        cracked_brick2 = cracked_brick.copy()
        cracked_brick2.paste(plain_full_block_crack, (0,0), plain_full_block_crack)
        # just the cracks, mamm
        
        if color in ['black', 'blue', 'brown', 'purple', 'red', 'gray']:
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .25)
        
        elif color in ['green', 'gray']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .30)
        
        elif color in ['cyan', 'green','light_blue', 'magenta', 'orange', 'pink', 'lime']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .45)
        
        elif color in ['pink', 'yellow', 'light_gray']:    
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.lighten_only, .95)
        else:
            cracked_brick = img_blend(cracked_brick, cracked_brick2, blend_modes.darken_only, .35)
            #cracked_brick = cracked_brick2
        
        add_it(cracked_brick, f"{name}_cracked{file_end}")
        
        cracked_brick.save(f"assets_out/{mod_name}/textures/block/etc/{name}_cracked{file_end}.png")
        
        #add_it(cracked_brick2, f"2{name}_cracked{file_end}")
        #add_it(big_brick(cracked_brick, big_brick_grout), f"{name}_cracked_big_bricks{file_end}")
    

#ipyplot.plot_images(bi_out)
ipyplot.plot_class_representations(bi_out, bn_out)

In [477]:
ipyplot.plot_class_representations(cracks_out, cracknames_out)

In [456]:
ipyplot.plot_images([Img.new('RGB', (16, 16), _rgb) for _rgb in chips_out], chipn_out)


In [434]:
whatever_tuple =  (108, 80, 134)

ipyplot.plot_images([Img.new('RGB', (16, 16), whatever_tuple)])

In [440]:
# print(rgb,get_color_ramp(base_rgb=c.get_rgb(), steps=2, light_lum=0.85, dark_lum=0.15))
steps = 2
rgb_tup =  (108, 80, 134)
r,g,b = rgb_tup
c = Color(rgb=(r/255, g/255, b/255))
light_lum=0.85 
dark_lum= 0.2
light_tone = Color(hue=c.get_hue(), saturation=c.get_saturation(), luminance=light_lum)
dark_tone = Color(hue=c.get_hue(), saturation=c.get_saturation(), luminance=dark_lum)

ramp_list = [tone.get_rgb() for tone in dark_tone.range_to(light_tone, steps)]

_ramp_list = [(int(_r * 255), int(_g * 255), int(_b * 255)) for (_r,_g,_b) in 
             [tone.get_rgb() for tone in dark_tone.range_to(light_tone, steps)]]
        

In [441]:
color_ramp = [Img.new('RGB', (16, 16), _rgb) for _rgb in 
              [(int(_r * 255), int(_g * 255), int(_b * 255)) for (_r,_g,_b) in 
             [tone.get_rgb() for tone in dark_tone.range_to(light_tone, steps)]]]
color_ramp.append(Img.new('RGB', (16, 16), rgb_tup))
ipyplot.plot_images(color_ramp)


In [457]:

new_c_r = get_color_ramp(steps=2,base_rgb=rgb_tup, light_lum=.85, dark_lum= .2)
test_ramper = [Img.new('RGB', (16, 16), dis_tup) for dis_tup in new_c_r]

ipyplot.plot_images(test_ramper)

0.10980392156862745 0.4196078431372549 0.7782679738562092
